**Set environment**

In [2]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
suppressMessages(suppressWarnings(library("DESeq2")))
suppressMessages(suppressWarnings(library("edgeR")))
show_env()

You are working on        Singularity 
BASE DIRECTORY (FD_BASE): /mount 
REPO DIRECTORY (FD_REPO): /mount/repo 
WORK DIRECTORY (FD_WORK): /mount/work 
DATA DIRECTORY (FD_DATA): /mount/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /mount/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /mount/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /mount/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /mount/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /mount/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /mount/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /mount/repo/Proj_ENCODE_FCC/log 
PROJECT APP     (FD_APP): /mount/repo/Proj_ENCODE_FCC/app 
PROJECT REF     (FD_REF): /mount/repo/Proj_ENCODE_FCC/references 



In [3]:
txt_fdiry = file.path(FD_RES, "region_coverage_fcc")
vec_txt_region = "fcc_astarr_macs_input_overlap" #dir(txt_fdiry)
vec_txt_assay  = c(
    "STARR_ATAC_K562_Reddy_KS274"
)

In [41]:
txt_region = "fcc_astarr_macs_input_union"
txt_assay  = "STARR_ATAC_K562_Reddy_KS91"

txt_fdiry = file.path(
    FD_RES, "region_coverage_fcc",
    txt_region,
    txt_assay,
    "overlap_count",
    "summary"
)
txt_fname = "data_list_count_column.rds"
txt_fpath = file.path(txt_fdiry, txt_fname)

txt_fpath

[1] "/mount/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_union/STARR_ATAC_K562_Reddy_KS91/overlap_count/summary/data_list_count_column.rds"

In [42]:
### read object
lst_dat = readRDS(txt_fpath)
dat_cnt = lst_dat$data_cnt
dat_col = lst_dat$data_col

### arrange count matrix
mat_cnt = dat_cnt %>%
    dplyr::select(-Chrom, -ChromStart, -ChromEnd) %>%
    column_to_rownames(var = "Region")

### arrange column data
dat_col = dat_col  %>% 
    #dplyr::select(Sample, Group) %>% 
    dplyr::mutate(Group = factor(Group, levels = c("Input", "Output"))) %>%
    column_to_rownames(var = "Sample")

In [43]:
head(mat_cnt)

,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Input.rep5,Input.rep6,Output.rep1,Output.rep2,Output.rep3,Output.rep4
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
chr1:10015-10442,155,214,257,236,185,214,7,12,4,11
chr1:14253-14645,110,144,160,141,130,130,8,26,30,57
chr1:16015-16477,141,208,206,190,202,182,9,9,18,23
chr1:17237-17772,259,350,399,367,369,331,7,13,23,59
chr1:28903-29613,263,338,368,333,352,317,12,18,3,32
chr1:30803-31072,82,115,171,136,105,115,13,22,14,33


In [44]:
dat_col

,FName,Assay,Prefix,Group
,<chr>,<chr>,<chr>,<fct>
Input.rep1,ASTARRseq_K562_KS91.hg38.Input.rep1.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input
Input.rep2,ASTARRseq_K562_KS91.hg38.Input.rep2.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input
Input.rep3,ASTARRseq_K562_KS91.hg38.Input.rep3.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input
Input.rep4,ASTARRseq_K562_KS91.hg38.Input.rep4.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input
Input.rep5,ASTARRseq_K562_KS91.hg38.Input.rep5.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input
Input.rep6,ASTARRseq_K562_KS91.hg38.Input.rep6.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input
Output.rep1,ASTARRseq_K562_KS91.hg38.Output.rep1.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Output
Output.rep2,ASTARRseq_K562_KS91.hg38.Output.rep2.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Output
Output.rep3,ASTARRseq_K562_KS91.hg38.Output.rep3.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Output


In [51]:
dge = DGEList(counts=mat_cnt, group=dat_col$Group)
#dge = DGEList(counts=mat_cnt)
mat = dge$counts
print(dim(mat))

[1] 246852     10


In [53]:
vec = apply(mat, 1, sum)
table(vec < 10)


 FALSE   TRUE 
246850      2 

In [58]:
vec = apply(mat, 1, sum)
idx = vec < 100
table(idx)

idx
 FALSE   TRUE 
246850      2 

In [59]:
vec[idx]

chr21:8219783-8221061 chr21:8396525-8397035 
                    8                     1

In [50]:
idx = filterByExpr(dge)
dge = dge[idx, , keep.lib.sizes=FALSE]
mat = dge$counts
print(dim(mat))

[1] 246850     10


In [ ]:
dge = DGEList(counts=mat_cnt, group=dat_col$Group)
idx = filterByExpr(dge)